In [96]:
# Each line after the first is either broad, medium, or narrow.
# Also sum for each industry.
# For the first attempt, just ignore the non-national data.
# Descending order of size: Major, Minor, Broad, Detail.
# For NAICS values, all are either two-digit or six digit, right-padded with 0s.
import csv
import re
with open("all_data_M_2015.csv",'r') as f:
    c=csv.reader(f)
    L=list(c)
print(L[0])

['area', 'area_title', 'area_type', 'naics', 'naics_title', 'own_code', 'occ code', 'occ title', 'group', 'tot_emp', 'emp_prse', 'jobs_1000', 'loc_quotient', 'pct_total', 'h_mean', ' a_mean ', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', ' a_pct10 ', ' a_pct25 ', ' a_median ', ' a_pct75 ', ' a_pct90 ', 'annual', 'hourly', '']


In [97]:
d=dict()
def get(i,count):
    return [j[count] for j in i[1:]]
d['naics']= get(l,3)     #naics
d['occ_code']= get(l,6)  #occ code, adjusted name here for consistency
d['group']= get(l,8)     #Group
d['tot_emp']= get(l,9)     #Total employment
d['h_mean']=get(l,14)    #Mean Hourly Pay
d['a_mean']=get(l,15)    #Mean Annual Pay, adjusted the name here to remove extraneous spaces
d["h_pay_dist"]=get(l,slice(17,22)) #10th, 25th, 50th, 75th, and 90th percentile hourly pay.
d["a_pay_dist"]=get(l,slice(22,27)) #10th, 25th, 50th, 75th, and 90th percentile annual pay.
all_jobs=[[d['naics'][n],d['occ_code'][n],d['group'][n],d['tot_emp'][n],d['h_mean'][n],d['a_mean'][n],
          d["h_pay_dist"][n],d["h_pay_dist"][n]] for n in range(len(d['naics']))]
d.keys()

dict_keys(['naics', 'occ_code', 'group', 'tot_emp', 'h_mean', 'a_mean', 'h_pay_dist', 'a_pay_dist'])

In [98]:
def select(func,universe_list=all_jobs):
    final=list()
    for i in universe_list:
        if func(i):
            final.append(i)
    return final

def match(index,value):
    return lambda x:(x[index]==value)

def access_job(occ_code,universe_dict=d,universe_list=all_jobs):
    code_list=universe_dict['occ_code']
    holder=[]
    for i,q in enumerate(code_list):
        if q==occ_code:
            holder.append(universe_list[i])
    return holder
#access_job("13-2000")

In [99]:
def get_percent(job_num,naics_codes): #job_num should be a string, naics_code a list of relevant naics codes.
    # Also works fine if you just use a single string.
    job_rows=access_job(str(job_num))
    denominator=0
    for row in job_rows:
        if row[0] in naics_codes:
            numerator=int(re.sub(",","",row[3]))
        try:
            denominator+=int(re.sub(",","",row[3]))
        except ValueError:
            #print(row[3])
            pass
    return numerator/denominator
get_percent("13-2000","541200")
def get_highest(job_num):
    industry_list=[a for a in sorted(set(d['naics']))[2:] if len(a)==6 and a[3]=='0']
    highest,denominator=0,0
    all_job_rows=access_job(str(job_num)) #Gets all rows that contain the job.
    rows=[a for a in all_job_rows if a[0] in industry_list] #Narrows it to exclude over-broad categories.
    for row in rows:
        try:
            num=int(re.sub(",","",row[3]))
        except ValueError:
            print(row)
            continue
        if num>highest:
            highest=num
            highest_row=row
        denominator+=int(re.sub(",","",row[3]))
    return highest_row,highest/denominator
# Financial Specialists and Accounting, Tax Preparation, Bookkeeping, and Payroll Services

In [107]:
job_list=[a for a in sorted(set(d['occ_code'])) if a[-1]!='0']
x=[[get_highest(job),job] for job in job_list]

['523000', '11-1011', 'detail ', '**', '116.63', '242,590', ['58.15', '89.79', '#', '#', '#'], ['58.15', '89.79', '#', '#', '#']]
['332000', '11-2011', 'detail ', '**', '45.44', '94,520', ['32.82', '39.52', '43.19', '46.86', '55.87'], ['32.82', '39.52', '43.19', '46.86', '55.87']]
['532000', '11-2011', 'detail ', '**', '45.20', '94,010', ['22.46', '31.80', '41.76', '57.55', '68.34'], ['22.46', '31.80', '41.76', '57.55', '68.34']]
['533000', '11-2011', 'detail ', '**', '53.50', '111,270', ['22.39', '30.53', '36.59', '70.34', '#'], ['22.39', '30.53', '36.59', '70.34', '#']]
['451000', '11-2021', 'detail ', '**', '66.68', '138,700', ['20.34', '22.45', '50.36', '#', '#'], ['20.34', '22.45', '50.36', '#', '#']]
['446000', '11-2022', 'detail ', '**', '46.15', '96,000', ['21.36', '27.41', '38.89', '57.46', '80.02'], ['21.36', '27.41', '38.89', '57.46', '80.02']]
['533000', '11-2031', 'detail ', '**', '49.31', '102,570', ['30.80', '37.10', '46.89', '64.09', '74.10'], ['30.80', '37.10', '46.89'

['323000', '13-2041', 'detail ', '**', '30.58', '63,610', ['17.20', '20.27', '25.38', '35.53', '54.76'], ['17.20', '20.27', '25.38', '35.53', '54.76']]
['481000', '13-2041', 'detail ', '**', '23.79', '49,480', ['8.36', '9.36', '20.01', '34.77', '50.52'], ['8.36', '9.36', '20.01', '34.77', '50.52']]
['441000', '13-2051', 'detail ', '**', '36.64', '76,210', ['28.82', '32.56', '36.17', '40.87', '47.39'], ['28.82', '32.56', '36.17', '40.87', '47.39']]
['521000', '13-2051', 'detail ', '**', '44.08', '91,690', ['24.22', '29.94', '38.70', '52.43', '71.22'], ['24.22', '29.94', '38.70', '52.43', '71.22']]
['525000', '13-2051', 'detail ', '**', '48.29', '100,450', ['22.40', '27.11', '37.10', '56.88', '#'], ['22.40', '27.11', '37.10', '56.88', '#']]
['623000', '13-2051', 'detail ', '**', '32.51', '67,630', ['20.67', '25.79', '31.63', '37.35', '47.31'], ['20.67', '25.79', '31.63', '37.35', '47.31']]
['531000', '13-2061', 'detail ', '**', '32.50', '67,590', ['17.53', '24.02', '34.23', '37.59', '39.

['512000', '17-2072', 'detail ', '**', '47.98', '99,800', ['33.50', '39.68', '47.30', '56.86', '63.37'], ['33.50', '39.68', '47.30', '56.86', '63.37']]
['238000', '17-2081', 'detail ', '**', '42.00', '87,360', ['21.80', '31.88', '38.30', '54.30', '66.11'], ['21.80', '31.88', '38.30', '54.30', '66.11']]
['425000', '17-2081', 'detail ', '**', '50.58', '105,220', ['38.54', '40.79', '44.43', '48.07', '72.90'], ['38.54', '40.79', '44.43', '48.07', '72.90']]
['441000', '17-2112', 'detail ', '**', '35.12', '73,040', ['20.28', '22.58', '28.32', '44.48', '66.47'], ['20.28', '22.58', '28.32', '44.48', '66.47']]
['213000', '17-2121', 'detail ', '**', '59.61', '123,990', ['32.60', '43.02', '57.12', '74.25', '#'], ['32.60', '43.02', '57.12', '74.25', '#']]
['441000', '17-2141', 'detail ', '**', '39.62', '82,400', ['24.83', '31.09', '38.89', '46.16', '60.22'], ['24.83', '31.09', '38.89', '46.16', '60.22']]
['721000', '17-2199', 'detail ', '**', '37.26', '77,500', ['18.81', '25.86', '34.18', '50.53',

['551000', '19-4051', 'detail ', '**', '37.41', '77,810', ['22.42', '29.74', '39.64', '44.51', '49.16'], ['22.42', '29.74', '39.64', '44.51', '49.16']]
['561000', '19-4051', 'detail ', '**', '35.22', '73,270', ['16.39', '19.18', '34.02', '48.88', '56.34'], ['16.39', '19.18', '34.02', '48.88', '56.34']]
['238000', '19-4091', 'detail ', '**', '22.59', '46,980', ['15.62', '19.72', '22.39', '26.38', '29.85'], ['15.62', '19.72', '22.39', '26.38', '29.85']]
['425000', '19-4091', 'detail ', '**', '26.93', '56,000', ['16.58', '18.86', '28.41', '32.95', '36.15'], ['16.58', '18.86', '28.41', '32.95', '36.15']]
['113000', '19-4093', 'detail ', '**', '20.31', '42,240', ['17.04', '19.37', '20.80', '22.23', '23.08'], ['17.04', '19.37', '20.80', '22.23', '23.08']]
['812000', '21-1011', 'detail ', '**', '12.80', '26,630', ['10.08', '11.32', '12.77', '14.06', '15.49'], ['10.08', '11.32', '12.77', '14.06', '15.49']]
['541000', '21-1013', 'detail ', '**', '22.82', '47,460', ['17.55', '20.28', '22.60', '2

['424000', '27-1019', 'detail ', '**', '35.52', '73,880', ['24.65', '29.32', '34.78', '39.94', '48.76'], ['24.65', '29.32', '34.78', '39.94', '48.76']]
['712000', '27-1019', 'detail ', '**', '24.01', '49,940', ['12.75', '15.52', '18.39', '33.77', '43.83'], ['12.75', '15.52', '18.39', '33.77', '43.83']]
['213000', '27-1021', 'detail ', '**', '45.80', '95,270', ['38.02', '40.97', '46.06', '52.70', '56.86'], ['38.02', '40.97', '46.06', '52.70', '56.86']]
['236000', '27-1021', 'detail ', '**', '27.63', '57,480', ['24.90', '25.99', '27.82', '29.64', '30.74'], ['24.90', '25.99', '27.82', '29.64', '30.74']]
['441000', '27-1021', 'detail ', '**', '21.01', '43,700', ['15.40', '16.30', '17.79', '21.03', '35.56'], ['15.40', '16.30', '17.79', '21.03', '35.56']]
['442000', '27-1021', 'detail ', '**', '25.60', '53,250', ['12.93', '14.44', '17.39', '27.50', '56.88'], ['12.93', '14.44', '17.39', '27.50', '56.88']]
['316000', '27-1022', 'detail ', '**', '29.12', '60,570', ['15.68', '20.11', '25.24', '3

...

['337000', '51-2091', 'detail ', '**', '15.11', '31,430', ['9.82', '12.22', '14.15', '17.42', '22.17'], ['9.82', '12.22', '14.15', '17.42', '22.17']]
['315000', '51-2092', 'detail ', '**', '11.99', '24,950', ['8.64', '9.95', '11.28', '13.56', '16.56'], ['8.64', '9.95', '11.28', '13.56', '16.56']]
['446000', '51-2092', 'detail ', '**', '15.62', '32,480', ['10.10', '12.39', '14.53', '18.63', '22.98'], ['10.10', '12.39', '14.53', '18.63', '22.98']]
['451000', '51-2092', 'detail ', '**', '13.03', '27,100', ['9.94', '10.51', '11.46', '13.61', '18.70'], ['9.94', '10.51', '11.46', '13.61', '18.70']]
['484000', '51-2092', 'detail ', '**', '12.57', '26,150', ['9.90', '10.71', '12.15', '13.96', '15.60'], ['9.90', '10.71', '12.15', '13.96', '15.60']]
['541000', '51-2093', 'detail ', '**', '21.62', '44,970', ['18.75', '20.26', '21.91', '23.55', '24.55'], ['18.75', '20.26', '21.91', '23.55', '24.55']]
['448000', '51-2099', 'detail ', '**', '12.89', '26,810', ['8.88', '9.83', '11.16', '13.33', '25.1

['323000', '51-4199', 'detail ', '**', '14.28', '29,700', ['12.03', '12.65', '13.69', '14.73', '18.81'], ['12.03', '12.65', '13.69', '14.73', '18.81']]
['333000', '51-5111', 'detail ', '**', '17.13', '35,640', ['9.98', '11.03', '17.16', '22.01', '26.00'], ['9.98', '11.03', '17.16', '22.01', '26.00']]
['315000', '51-5112', 'detail ', '**', '11.79', '24,520', ['8.72', '9.20', '10.62', '13.46', '17.41'], ['8.72', '9.20', '10.62', '13.46', '17.41']]
['812000', '51-5112', 'detail ', '**', '13.42', '27,920', ['9.28', '10.03', '11.19', '14.87', '22.70'], ['9.28', '10.03', '11.19', '14.87', '22.70']]
['326000', '51-5113', 'detail ', '**', '16.48', '34,280', ['9.74', '10.89', '14.93', '21.94', '27.28'], ['9.74', '10.89', '14.93', '21.94', '27.28']]
['551000', '51-5113', 'detail ', '**', '13.41', '27,880', ['9.57', '10.19', '11.21', '15.23', '22.06'], ['9.57', '10.19', '11.21', '15.23', '22.06']]
['236000', '51-6011', 'detail ', '**', '10.49', '21,820', ['8.52', '8.79', '9.22', '11.64', '14.04']

['493000', '51-8093', 'detail ', '**', '30.81', '64,070', ['19.82', '22.93', '31.37', '37.29', '43.99'], ['19.82', '22.93', '31.37', '37.29', '43.99']]
['561000', '51-8093', 'detail ', '**', '25.86', '53,790', ['15.88', '19.65', '24.10', '29.68', '41.73'], ['15.88', '19.65', '24.10', '29.68', '41.73']]
['423000', '51-8099', 'detail ', '**', '21.60', '44,930', ['14.74', '19.71', '21.88', '24.03', '27.81'], ['14.74', '19.71', '21.88', '24.03', '27.81']]
['334000', '51-9011', 'detail ', '**', '18.64', '38,770', ['12.38', '13.31', '14.89', '22.80', '30.16'], ['12.38', '13.31', '14.89', '22.80', '30.16']]
['326000', '51-9012', 'detail ', '**', '16.90', '35,150', ['9.00', '10.86', '15.18', '19.37', '29.30'], ['9.00', '10.86', '15.18', '19.37', '29.30']]
['327000', '51-9012', 'detail ', '**', '19.91', '41,410', ['10.06', '15.08', '20.68', '25.56', '28.48'], ['10.06', '15.08', '20.68', '25.56', '28.48']]
['423000', '51-9012', 'detail ', '**', '18.99', '39,500', ['11.67', '13.46', '16.48', '21.

['425000', '51-9195', 'detail ', '**', '14.16', '29,460', ['10.02', '10.54', '11.40', '12.83', '17.45'], ['10.02', '10.54', '11.40', '12.83', '17.45']]
['551000', '51-9195', 'detail ', '**', '15.50', '32,230', ['9.85', '12.26', '14.56', '18.80', '22.49'], ['9.85', '12.26', '14.56', '18.80', '22.49']]
['711000', '51-9195', 'detail ', '**', '12.28', '25,550', ['9.56', '10.07', '10.90', '11.74', '18.17'], ['9.56', '10.07', '10.90', '11.74', '18.17']]
['425000', '51-9196', 'detail ', '**', '12.53', '26,060', ['9.12', '9.33', '12.61', '14.62', '17.09'], ['9.12', '9.33', '12.61', '14.62', '17.09']]
['551000', '51-9196', 'detail ', '**', '16.58', '34,500', ['10.21', '14.40', '16.85', '19.34', '22.45'], ['10.21', '14.40', '16.85', '19.34', '22.45']]
['423000', '51-9197', 'detail ', '**', '13.99', '29,100', ['10.60', '11.92', '13.76', '16.28', '18.09'], ['10.60', '11.92', '13.76', '16.28', '18.09']]
['441000', '51-9198', 'detail ', '**', '13.49', '28,060', ['9.31', '10.49', '12.83', '14.95', '2

['999000', '53-5011', 'detail ', '**', '21.60', '44,930', ['15.80', '18.99', '22.55', '24.16', '25.63'], ['15.80', '18.99', '22.55', '24.16', '25.63']]
['311000', '53-5021', 'detail ', '**', '35.93', '74,740', ['26.12', '31.81', '35.37', '38.85', '50.69'], ['26.12', '31.81', '35.37', '38.85', '50.69']]
['562000', '53-5021', 'detail ', '**', '46.65', '97,030', ['25.07', '27.63', '46.26', '67.24', '71.93'], ['25.07', '27.63', '46.26', '67.24', '71.93']]
['311000', '53-5031', 'detail ', '**', '27.11', '56,390', ['20.54', '22.54', '25.80', '29.31', '40.24'], ['20.54', '22.54', '25.80', '29.31', '40.24']]
['532000', '53-5031', 'detail ', '**', '31.23', '64,960', ['17.16', '21.07', '27.98', '38.81', '48.09'], ['17.16', '21.07', '27.98', '38.81', '48.09']]
['236000', '53-6011', 'detail ', '**', '23.98', '49,890', ['13.52', '16.43', '25.81', '28.78', '30.56'], ['13.52', '16.43', '25.81', '28.78', '30.56']]
['445000', '53-6021', 'detail ', '**', '11.44', '23,800', ['8.40', '9.30', '10.94', '13.

In [113]:
get_highest("15-1100")
x[0][0][1]

0.1298273893478407

In [124]:
import matplotlib
matplotlib.style.use('ggplot')
import pandas as pd
import numpy as np

df=pd.DataFrame(a[0][1] for a in x)
S=pd.Series(a[0][1] for a in x)

In [128]:
S=S.cumsum()
ts = ts.cumsum()
matplotlib.pyplot.scatter([a[0][1] for a in x],range(len(x)))